In [38]:
pip install bertviz

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [8]:
from transformers import BertTokenizer, BertModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode some input text
input_text = "The course on LLMs is great"
inputs = tokenizer(input_text, return_tensors="pt")

# Load pre-trained BERT model
model = BertModel.from_pretrained("bert-base-uncased")

# Forward pass through the model
outputs = model(**inputs)

# Get the last hidden states from the model
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states)


tensor([[[-0.1007,  0.0123,  0.3084,  ...,  0.0450,  0.0946,  0.3813],
         [-0.7610, -0.4895,  0.5019,  ...,  0.1161,  0.3706, -0.0689],
         [-0.1832,  0.0089,  0.8181,  ...,  0.0775, -0.2027, -0.3375],
         ...,
         [-0.6238, -0.6102,  0.2817,  ...,  0.1255,  0.0713,  0.5276],
         [-0.2435, -0.4879,  0.1014,  ...,  0.5436,  0.0487,  0.0971],
         [ 0.8228,  0.0720, -0.1718,  ...,  0.3141, -0.7593, -0.1244]]],
       grad_fn=<NativeLayerNormBackward0>)


In [11]:
last_hidden_states = []

def hidden_state(string_input):
    # Load pre-trained model tokenizer (vocabulary)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Load pre-trained BERT model (loading outside the loop for efficiency)
    model = BertModel.from_pretrained("bert-base-uncased")
    
    # Split the input text into sentences
    input_text = string_input
    sentences = input_text.split('.')
    
    # Process each sentence
    for sentence in sentences:
        # Encode the sentence (add a check for non-empty sentences)
        if sentence.strip():
            inputs = tokenizer(sentence.strip(), return_tensors="pt")
            
            # Forward pass through the model
            outputs = model(**inputs)
            
            # Append the last hidden states to the list
            last_hidden_states.append(outputs.last_hidden_state)
    
    return last_hidden_states




In [12]:
# Example usage:
string_input = "Generative AI is transforming the future of NLP. It is being adopted widely."
hidden_states_output = hidden_state(string_input)
print(hidden_states_output)

[tensor([[[-0.2231, -0.0833, -0.0051,  ..., -0.6667, -0.0291,  0.5422],
         [-0.8698,  0.3924, -0.0981,  ..., -0.3119,  0.4725,  0.5823],
         [-1.2071, -0.0288,  0.5067,  ..., -0.4567, -0.5583,  0.0859],
         ...,
         [ 0.1181, -1.1284,  0.1730,  ..., -0.5175,  0.8619,  0.2247],
         [-0.9253, -0.3118, -0.2318,  ..., -0.3344, -0.4574, -0.0069],
         [ 0.4301, -0.0257, -0.1754,  ..., -0.1605, -0.8548, -0.2293]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[-0.1432,  0.0047,  0.3337,  ..., -0.1060,  0.2802,  0.4039],
         [ 0.0426, -0.3422,  0.0814,  ..., -0.5513, -0.2363,  0.3278],
         [-0.0841, -0.1485,  0.1901,  ..., -0.0854,  0.2737,  0.1977],
         ...,
         [ 0.3336,  0.3924,  0.0189,  ..., -0.4522,  0.5630,  0.4153],
         [-0.1472, -0.7131,  0.1172,  ...,  0.1742,  0.5668, -0.0437],
         [ 0.6686,  0.1506,  0.0948,  ..., -0.1397, -0.7206, -0.4086]]],
       grad_fn=<NativeLayerNormBackward0>)]


In [30]:
from transformers import pipeline

# Initialize the question-answering pipeline
qa_pipeline = pipeline('question-answering')

# Define a context paragraph
context = """
Khan's first starring role was in Lekh Tandon's television series Dil Dariya, which began shooting in 1988, 
but production delays led to the Raj Kumar Kapoor directed 1989 series Fauji becoming his television debut instead.[37] 
In the series, which depicted a realistic look at the training of army cadets, he played the leading role of Abhimanyu Rai.[38][39] 
This led to further appearances in Aziz Mirza's television series Circus (1989–90) and Mani Kaul's miniseries Idiot (1992).[40] 
Khan also played minor parts in the serials Umeed (1989) and Wagle Ki Duniya (1988–90),[40] and in the English-language television film In 
Which Annie Gives It Those Ones (1989).[41] His appearances in these serials led critics to compare his look and acting style with those of the film actor 
Dilip Kumar,[42] but Khan was not interested in film acting at the time, thinking that he was not good enough.[40][43]
"""

# Define a question
question = "What was khan's 1st television appearance?"

# Get the answer
result = qa_pipeline(question=question, context=context)

print("Answer:", result['answer'])


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Answer: Dil Dariya


In [37]:
def answer_questions(context, questions):
    results = []
    qa_pipeline = pipeline(
        'question-answering', 
        model='distilbert-base-cased-distilled-squad',
        tokenizer='distilbert-base-cased-distilled-squad'
        )

    for question in questions:
        result = qa_pipeline(question=question, context=context)
        results.append({'question': question, 'answer': result['answer'], 'score': result['score']})
        
    return results

# Example usage
context = """
LangChain is a framework for developing applications powered by language models.
It enables the creation of complex applications by chaining together different components.
"""
questions = [
    "What is LangChain?",
    "What does LangChain enable?",
    "How does LangChain help developers?"
]

answers = answer_questions(context, questions)
for item in answers:
    if item['score']<0.1:
        print(f"Question: {item['question']}\nAnswer not found\n")
    else:
        print(f"Question: {item['question']}\nAnswer: {item['answer']}\nScore: {item['score']}\n")


Question: What is LangChain?
Answer: a framework for developing applications powered by language models
Score: 0.6474099159240723

Question: What does LangChain enable?
Answer: the creation of complex applications
Score: 0.20924898982048035

Question: How does LangChain help developers?
Answer: by chaining together different components
Score: 0.2618699073791504



In [39]:
from transformers import BertTokenizer, BertModel
from bertviz import head_view

model = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentence = "The quick brown fox jumps over the lazy dog"
inputs = tokenizer.encode(sentence, return_tensors='pt')
outputs = model(inputs)

attention = outputs.attentions  # Get attentions from all layers
head_view(attention, tokenizer.convert_ids_to_tokens(inputs[0]))


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


<IPython.core.display.Javascript object>

In [41]:
import cohere

# Replace 'your-api-key' with your actual API key
COHERE_API_KEY = "7jDhE7Cke10ET0UPPXy7p0GaFqFVSLb7KCYVwyy1"
co = cohere.Client(COHERE_API_KEY)

In [42]:
# Example context and question
context = """
LangChain is a framework designed for building language model applications. It allows users to combine various 
language model tasks such as question answering, summarization, and text generation into workflows.
"""
query = "What is LangChain?"

# Query Cohere with your context and question
response = co.generate(
    prompt=f"Context: {context}\n\nQuestion: {query}\nAnswer:",
    max_tokens=50,
    temperature=0.7,
)

# Output the generated answer
print(f"Answer: {response.generations[0].text.strip()}")


Answer: LangChain is a framework that facilitates the construction of applications for language models. It empowers users to merge different language tasks like question answering, text summarization, and text generation into workflows.


In [43]:
response

Generation(id='94b77552-2244-4121-8b88-be1f22c10daa', prompt='Context: \nLangChain is a framework designed for building language model applications. It allows users to combine various \nlanguage model tasks such as question answering, summarization, and text generation into workflows.\n\n\nQuestion: What is LangChain?\nAnswer:', generations=[SingleGeneration(id='35bff82d-6e58-4358-8eed-734fa4df9105', text=' LangChain is a framework that facilitates the construction of applications for language models. It empowers users to merge different language tasks like question answering, text summarization, and text generation into workflows. ', index=None, likelihood=None, token_likelihoods=None, finish_reason='COMPLETE')], meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=53, output_tokens=39, search_units=None, classifications=None), tokens=None, warnings=None))

In [45]:
# Simple in-memory document retrieval system
documents = [
    {"id": "1", "text": "LangChain is a framework for building LLM applications."},
    {"id": "2", "text": "LangChain enables chaining tasks together."},
]

def retrieve(query):
    # A basic retrieval based on keyword matching
    return [doc for doc in documents if query.lower() in doc['text'].lower()]

# Example query
query = "What is LangChain?"
retrieved_docs = retrieve(query)
retrieved_docs


[]

In [46]:
# Retrieve the best match document
context = retrieved_docs[0]['text'] if retrieved_docs else "No relevant document found."

# Use Cohere to generate an answer based on the retrieved context
response = co.generate(
    prompt=f"Context: {context}\n\nQuestion: {query}\nAnswer:",
    max_tokens=50,
    temperature=0.7,
)

# Output the generated answer
print(f"Answer: {response.generations[0].text.strip()}")


Answer: No relevant document found.

Based on this, I cannot provide an answer to the question "What is LangChain?". 
However, I would be happy to answer any other questions you may have.
